In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
import os
from tqdm import tqdm

from konlpy.tag import Okt

import sklearn
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import log_loss, accuracy_score,f1_score
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from transformers import *


In [2]:
path_train = "1. 실습용자료.txt"
train =  pd.read_csv(path_train,sep="|",encoding='CP949')

path = "2. 모델개발용자료.txt"
test = pd.read_csv(path,sep="|",encoding='CP949')

In [3]:
train.shape

(1000000, 7)

In [4]:
train.fillna(" ", inplace = True)
test.fillna(" ", inplace = True)

In [5]:
train["text"] = ""
train["text"][train.index] = train["text_obj"][train.index] + " " + train["text_mthd"][train.index] + " " + train["text_deal"][train.index]
test["text"] = ""
test["text"][test.index] = test["text_obj"][test.index] + " " + test["text_mthd"][test.index] + " " + test["text_deal"][test.index]

<ipython-input-5-33263761baf3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["text"][train.index] = train["text_obj"][train.index] + " " + train["text_mthd"][train.index] + " " + train["text_deal"][train.index]


In [6]:
#random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)
BATCH_SIZE = 32
NUM_EPOCHS = 3
VALID_SPLIT = 0.2
MAX_LEN=30

In [7]:
tokenizer=BertTokenizer.from_pretrained('bert-base-multilingual-cased',  cache_dir='bert_ckpt', do_lower_case=False)

def bert_tokenizer(sent):
    
    encoded_dict=tokenizer.encode_plus(
    text = sent, 
    add_special_tokens=True, 
    max_length=MAX_LEN, 
    pad_to_max_length=True, 
    return_attention_mask=True,
    truncation = True)
    
    input_id=encoded_dict['input_ids']
    attention_mask = encoded_dict['attention_mask']
    token_type_id = encoded_dict['token_type_ids']
    
    return input_id, attention_mask, token_type_id

input_ids =[]
attention_masks =[]
token_type_ids =[]
train_data_labels = []

def clean_text(sent):
    sent_clean = re.sub("[^가-힣ㄱ-하-ㅣ]", "", sent)
    return sent_clean

for train_sent, train_label in zip(train['text'], train['digit_3']):
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(clean_text(train_sent))
        
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        #########################################
        train_data_labels.append(train_label)
        
    except Exception as e:
        print(e)
        print(train_sent)
        pass

train_input_ids=np.array(input_ids, dtype=int)
train_attention_masks=np.array(attention_masks, dtype=int)
train_token_type_ids=np.array(token_type_ids, dtype=int)
###########################################################
train_inputs=(train_input_ids, train_attention_masks, train_token_type_ids)
train_labels=np.asarray(train_data_labels, dtype=np.int32)

loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/vocab.txt from cache at bert_ckpt/eff018e45de5364a8368df1f2df3461d506e2a111e9dd50af1fae061cd460ead.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer_config.json from cache at bert_ckpt/f55e7a2ad4f8d0fff2733b3f79777e1e99247f2e4583703e92ce74453af8c235.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at bert_ckpt/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model conf

In [8]:
class TFBertClassifier(tf.keras.Model):
    def __init__(self, model_name, dir_path, num_class):
        super(TFBertClassifier, self).__init__()

        self.bert = TFBertModel.from_pretrained(model_name, cache_dir=dir_path)
        self.dropout = tf.keras.layers.Dropout(self.bert.config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(num_class, 
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(self.bert.config.initializer_range), 
                                                name="classifier")
        
    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):
        
        #outputs 값: # sequence_output, pooled_output, (hidden_states), (attentions)
        outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs[1] 
        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)

        return logits

cls_model = TFBertClassifier(model_name='bert-base-multilingual-cased',
                                  dir_path='bert_ckpt',
                                  num_class= 1000)


loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at bert_ckpt/6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cach

In [9]:
optimizer = tf.keras.optimizers.Adam(3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

cls_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

model_name = "tf2_bert_classifier"

# overfitting을 막기 위한 ealrystop 추가
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001,patience=5)

checkpoint_path = os.path.join(model_name, 'weights.h5')
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create path if exists
if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))
    
cp_callback = ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)


tf2_bert_classifier -- Folder already exists 



In [12]:
# history 먼저 돌리고 스탑한 뒤 불러들이고 다시 학습
cls_model.load_weights("/Users/jang-yeongdong/Desktop/경진대회/tf2_bert_classifier/weights.h5")

In [13]:
# 학습과 eval 시작
history = cls_model.fit(train_inputs, train_labels, epochs=30, batch_size=64,
                    validation_split = 0.2, callbacks=[earlystop_callback, cp_callback])

Epoch 1/30
    1/12500 [..............................] - ETA: 60:45:28 - loss: 0.3314 - accuracy: 0.8750

KeyboardInterrupt: 

In [36]:
train.head()

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환,카센터에서 자동차부분정비 타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매,상점내에서 일반인을 대상으로 채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매,절단하여사업체에도매 공업용고무를가지고 합성고무도매
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치,영업점에서 일반소비자에게 열쇠잠금장치
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육,어린이집 보호자의 위탁을 받아 취학전아동보육


In [10]:
input_ids =[]
attention_masks =[]
token_type_ids =[]
train_data_labels = []

def clean_text(sent):
    sent_clean=re.sub("[^가-힣ㄱ-하-ㅣ]", " ", sent)
    return sent_clean

for test_sent in test['text']:
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(clean_text(test_sent))
        
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        #########################################
       
    except Exception as e:
        print(e)
        print(test_sent)
        pass
    
test_input_ids=np.array(input_ids, dtype=int)
test_attention_masks=np.array(attention_masks, dtype=int)
test_token_type_ids=np.array(token_type_ids, dtype=int)
###########################################################
test_inputs=(test_input_ids, test_attention_masks, test_token_type_ids)

In [14]:
results = cls_model.predict(test_inputs)
# results=tf.argmax(results, axis=1)

In [15]:
results=tf.argmax(results, axis=1)

In [16]:
test['digit_3']=results


In [18]:
test.to_csv('bert_test_digit_3.csv',index=False,encoding="cp949")

In [24]:
len(test['digit_3'].unique())

199

In [51]:
for idx in test.index:
    test.loc[idx,'digit_2'] = str(test.loc[idx,'digit_3'])[:-1]
    test.loc[idx,'digit_2'] = int(test.loc[idx,'digit_2'])
    
    if 1 <= test.loc[idx,'digit_2'] <= 3:
        test.loc[idx,'digit_1'] = 'A'
    elif 5 <= test.loc[idx,'digit_2'] <= 8:
        test.loc[idx,'digit_1'] = 'B'
    elif 10 <=test.loc[idx,'digit_2'] <= 34:
        test.loc[idx,'digit_1'] = 'C'
    elif test.loc[idx,'digit_2'] == 35:
        test.loc[idx,'digit_1'] = 'D'
    elif 36 <=test.loc[idx,'digit_2'] <= 39:
        test.loc[idx,'digit_1'] = 'E'
    elif 41 <=test.loc[idx,'digit_2'] <= 42:
        test.loc[idx,'digit_1'] = 'F'
    elif 45 <=test.loc[idx,'digit_2'] <= 47:
        test.loc[idx,'digit_1'] = 'G'
    elif 49 <=test.loc[idx,'digit_2'] <= 52:
        test.loc[idx,'digit_1'] = 'H'
    elif 55 <=test.loc[idx,'digit_2'] <= 56:
        test.loc[idx,'digit_1'] = 'I'
    elif 58 <=test.loc[idx,'digit_2'] <= 63:
        test.loc[idx,'digit_1'] = 'J'
    elif 64 <=test.loc[idx,'digit_2'] <= 66:
        test.loc[idx,'digit_1'] = 'K'
    elif test.loc[idx,'digit_2'] == 68:
        test.loc[idx,'digit_1'] = 'L'
    elif 70 <=test.loc[idx,'digit_2'] <= 73:
        test.loc[idx,'digit_1'] = 'M'
    elif 74 <=test.loc[idx,'digit_2'] <= 76:
        test.loc[idx,'digit_1'] = 'N'
    elif test.loc[idx,'digit_2'] == 84:
        test.loc[idx,'digit_1'] = 'O'
    elif test.loc[idx,'digit_2'] == 85:
        test.loc[idx,'digit_1'] = 'P'
    elif 86 <=test.loc[idx,'digit_2'] <= 87:
        test.loc[idx,'digit_1'] = 'Q'
    elif 90 <=test.loc[idx,'digit_2'] <= 91:
        test.loc[idx,'digit_1'] = 'R'
    elif 94 <=test.loc[idx,'digit_2'] <= 96:
        test.loc[idx,'digit_1'] = 'S'
    elif 97 <=test.loc[idx,'digit_2'] <= 98:
        test.loc[idx,'digit_1'] = 'T'
    elif test.loc[idx,'digit_2'] == 99:
        test.loc[idx,'digit_1'] = 'U'

In [52]:
test.head()

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text
0,id_000001,I,56,561,치킨전문점에서,고객의주문에의해,치킨판매,치킨전문점에서 고객의주문에의해 치킨판매
1,id_000002,G,46,466,산업공구,다른 소매업자에게,철물 수공구,산업공구 다른 소매업자에게 철물 수공구
2,id_000003,S,94,949,절에서,신도을 대상으로,불교단체운영,절에서 신도을 대상으로 불교단체운영
3,id_000004,C,30,302,영업장에서,고객요구로,자동차튜닝,영업장에서 고객요구로 자동차튜닝
4,id_000005,I,56,562,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공","실내포장마차에서 접객시설을 갖추고 소주,맥주제공"


In [53]:
test.to_csv('test.csv',index=False,encoding="cp949")